In [36]:
import pandas as pd

In [52]:
customers = {
    'name': ['NTUC Income', 'GIC'],
    'since': [pd.Timestamp('19970314'), pd.Timestamp('20030120')],
    'signatories': [3, 6],
    'industry': ['Financial', 'Sovereign'],
    'requireKYC': [True, False],
    'aum': [41.9, 488]
}

print(customers)

{'name': ['NTUC Income', 'GIC'], 'since': [Timestamp('1997-03-14 00:00:00'), Timestamp('2003-01-20 00:00:00')], 'signatories': [3, 6], 'industry': ['Financial', 'Sovereign'], 'requireKYC': [True, False], 'aum': [41.9, 488]}


In [14]:
customers['industry']

['Financial', 'Sovereign']

In [18]:
## how do you check in a key is in the `customers`  dictionary?
## for example: check if 'account manager' is a valid key in that dictionary?
if 'aum' in customers:
    print('Found it!')

Found it!


In [55]:
customerdf = pd.DataFrame(customers)
print(customerdf)

          name      since  signatories   industry  requireKYC    aum
0  NTUC Income 1997-03-14            3  Financial        True   41.9
1          GIC 2003-01-20            6  Sovereign       False  488.0


In [23]:
customerdf['industry'] = customerdf['industry'].astype('category')

- `dtype` (singular form) works on one column and return the data type of that column.
- `dtypes`(plural) works on a dataframe and return the data types of all columns in that df

In [25]:
customerdf['industry'].dtype

CategoricalDtype(categories=['Financial', 'Sovereign'], ordered=False)

In [24]:
# check the data types of your `customers` dataframe
print(customerdf.dtypes)

name                   object
since          datetime64[ns]
signatories             int64
industry             category
requireKYC               bool
aum                   float64
dtype: object


In [38]:
customerdf.count()

name           2
since          2
signatories    2
industry       2
requireKYC     2
aum            2
dtype: int64

In [41]:
print(len(customerdf.index))
print(len(customerdf.columns))

2
6


In [33]:
# use the f-string method on customerdf, print: 'We have 2 customers in our database'
print(f'We have {customerdf.shape[0]} customers in our database')

We have 2 customers in our database


In [45]:
customerdf.columns

Index(['name', 'since', 'signatories', 'industry', 'requireKYC', 'aum'], dtype='object')

In [44]:
# .describe() looks at all NUMERIC columns in the dataframe, and give you the summary statistics
customerdf.describe()

,signatories,aum
count,2.00000,2.000000
mean,4.50000,264.950000
std,2.12132,315.440335
min,3.00000,41.900000
25%,3.75000,153.425000
50%,4.50000,264.950000
75%,5.25000,376.475000
max,6.00000,488.000000


In [32]:
# no computation is performed (a big giveaway is there is no method call)
print(customerdf.shape)
print(customerdf.shape[0] * customerdf.shape[1])
print(customerdf.size)

(2, 6)
12
12


#### Side Note: Where is `[]` and `()` being used in Python?

1. `[]` is used in indexing: `customers[:3]`, `customers['industry'][0]`
2. To create a list, we use `[]`: `balances = [0, 300, 29.9]`
3. `()` is used in method calls: `dbs.checkBalance()`, `dbs.addAccount()`
4. To create a tuple, we use `()`: `balances = (400, 10, 0)`

#### Side Note: Common exploratory operations on dataframes

```
creditscore = pd.read_csv("data_input/creditscore21.csv")

# inspect the first or last n rows of data
creditscore.head(3)
creditscore.tail(3)

# inspect the shape / structure of the data
creditscore.shape[0]
creditscore.size

# use .describe() to get a general sense of the distribution in our data
creditscore['loan_amount'].describe()
creditscore.describe()

# use .dtype (one column) and .dtypes (the entire df) to inspect the data types
creditscore.dtypes
creditscore['loan_duration'].dtype

# show all features in the dataframe
creditscore.columns
creditscore.index
'branchnumber' in creditscore.columns

# create a DataFrame out of a python dictionary
pd.DataFrame(our_dict)

# using pd.Timestamp() to create datetime columns
pd.Timestamp('20200328')

# perform simple type conversion using `.astype('int')`
creditscore['education_level'].astype('category')

# pd.read_csv to read in csv
aum = pd.read_csv(
        "asset_under_management.csv",
        index_col="asset",
        parse_dates=["acquisition_date", "last_updated", "record_date"]
)

# Using select dtypes to include or exclude specific columns based on types
aum.select_dtypes(exclude=['object'])

# Method chaining
pd.read_csv("asset_under_management.csv").select_dtypes(include=['int', 'float']).describe().head()
```

## Working with Loans

In [60]:
loans = pd.read_csv(
    "https://media.githubusercontent.com/media/onlyphantom/python4bankers/main/data_input/loan_new.csv",
    index_col='id',
    parse_dates=['issue_d']
)
print(loans.shape)

(887379, 29)


In [61]:
df = loans.copy()

In [71]:
# print the last 2 rows of the dataframe
loans.tail(2)

,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,loan_amount,...,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
id,,,,,,,,,,,,,,,,,,,,,
36490806,2015,2015-01-01,1012016,1.0,RENT,1,Low,64400,1,12000,...,0,19.99,E,5,27.19,3787.67,1552.69,0.0,317.86,Northern-Irl
36271262,2015,2015-01-01,1012016,10.0,RENT,1,Low,100000,1,20000,...,0,11.99,B,2,10.83,7943.76,5889.06,0.0,664.20,leinster


In [73]:
loans.head(10).describe().tail()

,year,final_d,emp_length_int,home_ownership_cat,annual_inc,income_cat,loan_amount,term_cat,application_type_cat,purpose_cat,interest_payment_cat,loan_condition_cat,interest_rate,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment
min,2011.0,1012015.00,0.50,1.0,12252.0,1.0,2400.00,1.0,1.0,1.00,1.0,0.00,7.90,1.0,1.0000,646.020000,162.020,0.00,59.8300
25%,2011.0,1012015.00,1.50,1.0,25500.0,1.0,3000.00,1.0,1.0,2.25,1.0,0.00,12.69,2.0,6.3425,1858.055911,1063.385,0.00,90.6050
50%,2011.0,1012016.00,6.00,1.0,38000.0,1.0,5000.00,1.5,1.0,3.50,2.0,0.00,14.38,3.0,14.5700,3590.157167,2700.000,0.00,136.9200
75%,2011.0,1042012.75,9.75,1.0,47751.0,1.0,5543.75,2.0,1.0,4.00,2.0,0.75,15.96,3.0,19.5200,5803.647999,5000.000,87.81,161.2675
max,2011.0,1112012.00,10.00,2.0,80000.0,1.0,10000.00,2.0,1.0,6.00,2.0,1.00,21.28,6.0,27.6500,12226.302210,10000.000,269.29,339.3100


In [64]:
# inspect the basic statistical summary of all numeric columns in our dataframe
x = loans.describe()
x

,year,final_d,emp_length_int,home_ownership_cat,annual_inc,income_cat,loan_amount,term_cat,application_type_cat,purpose_cat,interest_payment_cat,loan_condition_cat,interest_rate,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment
count,887379.000000,8.873790e+05,887379.000000,887379.000000,8.873790e+05,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000
mean,2014.021761,1.047089e+06,6.050564,2.099130,7.502759e+04,1.196702,14755.264605,1.300045,1.000576,4.874621,1.475629,0.075987,13.246740,2.798403,18.157039,7558.826684,5757.706424,45.919243,436.717127
std,1.261741,4.555149e+04,3.507405,0.944839,6.469815e+04,0.442542,8435.455601,0.458278,0.023990,2.381156,0.499406,0.264977,4.381867,1.312599,17.190626,7871.243336,6625.441046,409.693874,244.186593
min,2007.000000,1.012008e+06,0.500000,1.000000,0.000000e+00,1.000000,500.000000,1.000000,1.000000,1.000000,1.000000,0.000000,5.320000,1.000000,0.000000,0.000000,0.000000,0.000000,15.670000
25%,2013.000000,1.012016e+06,3.000000,1.000000,4.500000e+04,1.000000,8000.000000,1.000000,1.000000,3.000000,1.000000,0.000000,9.990000,2.000000,11.910000,1914.590000,1200.570000,0.000000,260.705000
50%,2014.000000,1.012016e+06,6.050000,3.000000,6.500000e+04,1.000000,13000.000000,1.000000,1.000000,6.000000,1.000000,0.000000,12.990000,3.000000,17.650000,4894.999117,3215.320000,0.000000,382.550000
75%,2015.000000,1.092015e+06,10.000000,3.000000,9.000000e+04,1.000000,20000.000000,2.000000,1.000000,6.000000,2.000000,0.000000,16.200000,4.000000,23.950000,10616.814230,8000.000000,0.000000,572.600000
max,2015.000000,1.122015e+06,10.000000,6.000000,9.500000e+06,3.000000,35000.000000,2.000000,2.000000,14.000000,2.000000,1.000000,28.990000,7.000000,9999.000000,57777.579870,35000.030000,33520.270000,1445.460000


In [74]:
# inspect the data types of your columns
loans.dtypes.head(5)

year                       int64
issue_d           datetime64[ns]
final_d                    int64
emp_length_int           float64
home_ownership            object
dtype: object

In [77]:
# get statistical summary on all float and integer
loans.select_dtypes(include=['float']).describe()

,emp_length_int,interest_rate,dti,total_pymnt,total_rec_prncp,recoveries,installment
count,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000
mean,6.050564,13.246740,18.157039,7558.826684,5757.706424,45.919243,436.717127
std,3.507405,4.381867,17.190626,7871.243336,6625.441046,409.693874,244.186593
min,0.500000,5.320000,0.000000,0.000000,0.000000,0.000000,15.670000
25%,3.000000,9.990000,11.910000,1914.590000,1200.570000,0.000000,260.705000
50%,6.050000,12.990000,17.650000,4894.999117,3215.320000,0.000000,382.550000
75%,10.000000,16.200000,23.950000,10616.814230,8000.000000,0.000000,572.600000
max,10.000000,28.990000,9999.000000,57777.579870,35000.030000,33520.270000,1445.460000


In [78]:
loans.select_dtypes(exclude=['object']).tail(3)

,year,issue_d,final_d,emp_length_int,home_ownership_cat,annual_inc,income_cat,loan_amount,term_cat,application_type_cat,purpose_cat,interest_payment_cat,loan_condition_cat,interest_rate,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment
id,,,,,,,,,,,,,,,,,,,,
36271333,2015,2015-01-01,1012016,5.0,1,35000,1,13000,2,1,6,2,0,15.99,4,30.90,3769.74,1844.50,0.0,316.07
36490806,2015,2015-01-01,1012016,1.0,1,64400,1,12000,2,1,6,2,0,19.99,5,27.19,3787.67,1552.69,0.0,317.86
36271262,2015,2015-01-01,1012016,10.0,1,100000,1,20000,1,1,1,1,0,11.99,2,10.83,7943.76,5889.06,0.0,664.20


In [66]:
x['interest_rate']

count    887379.000000
mean         13.246740
std           4.381867
min           5.320000
25%           9.990000
50%          12.990000
75%          16.200000
max          28.990000
Name: interest_rate, dtype: float64

In [76]:
customers = {
    'name': ['NTUC Income', 'GIC'],
    'since': [pd.Timestamp('19970314'), pd.Timestamp('20030120')],
    'signatories': [3, 6],
    'industry': ['Financial', 'Sovereign'],
    'requireKYC': [True, False],
    'aum': [41.9, 488],
    'pic': ['Janette@ntucincome.com.sg', 'Denise@gic.com']
}
customerdf = pd.DataFrame(customers)
backup = customerdf.copy()
backup

,name,since,signatories,industry,requireKYC,aum,pic
0,NTUC Income,1997-03-14,3,Financial,True,41.9,Janette@ntucincome.com.sg
1,GIC,2003-01-20,6,Sovereign,False,488.0,Denise@gic.com


In [77]:
backup = backup.drop(1)
backup

,name,since,signatories,industry,requireKYC,aum,pic
0,NTUC Income,1997-03-14,3,Financial,True,41.9,Janette@ntucincome.com.sg


In [85]:
backup = backup.drop(['pic', 'signatories'],
                     axis=1)
backup

,account,since,industry,requireKYC,aum
0,NTUC Income,1997-03-14,Financial,True,41.9


In [86]:
backup

,account,since,industry,requireKYC,aum
0,NTUC Income,1997-03-14,Financial,True,41.9


In [82]:
# rename 'name' column to 'account'
backup = backup.rename(
    columns={
        "name": "account"
    }
)

In [84]:
print(backup.columns[0])
print(backup.columns)

account
Index(['account', 'since', 'signatories', 'industry', 'requireKYC', 'aum',
       'pic'],
      dtype='object')


Copy, drop (rows / columns), rename, indexing and subsetting

In [50]:
x = [0, 2, 4]
print(id(x))
y = x.copy()
print(id(y))
z = [0, 2, 4]
print(id(z))

x.append(3)
print(len(x))
print(len(y))
print(z)

4767320768
4767250560
4767256064
4
3
[0, 2, 4]


In [56]:
customerdf

,name,since,signatories,industry,requireKYC,aum
0,NTUC Income,1997-03-14,3,Financial,True,41.9
1,GIC,2003-01-20,6,Sovereign,False,488.0


In [57]:
backup = customerdf.copy()
backup

,name,since,signatories,industry,requireKYC,aum
0,NTUC Income,1997-03-14,3,Financial,True,41.9
1,GIC,2003-01-20,6,Sovereign,False,488.0


In [58]:
backup['requireKYC'] = True
backup['signatories'] = 4
backup

,name,since,signatories,industry,requireKYC,aum
0,NTUC Income,1997-03-14,4,Financial,True,41.9
1,GIC,2003-01-20,4,Sovereign,True,488.0


In [59]:
customerdf.head(2)

,name,since,signatories,industry,requireKYC,aum
0,NTUC Income,1997-03-14,3,Financial,True,41.9
1,GIC,2003-01-20,6,Sovereign,False,488.0


In [16]:
from datetime import datetime, timezone, date

In [18]:
d = datetime.now()
d

datetime.datetime(2021, 6, 2, 7, 17, 56, 657725)

In [21]:
d.date().isoformat()

'2021-06-02'

- `%d` - date
- `%m` - month
- `%y` - year (two digit year)
- `%Y` - Year (four digit year)
- `%b` - month (written out, abbr to 3-letters)
- `%B` - month (written out, non-abbreviated)

In [33]:
d.date().strftime("%B %d %Y")

'June 02 2021'

In [4]:
from dateutil.parser import parse

date_array = [
    '2021-06-29 08:15:27.243860',
    'Jun 28 2018 7:40AM',
    'Jun 28 2018 at 7:40AM',
    'September 18, 2017, 22:19:55',
    'Sun, 05/12/1999, 12:30PM',
    'Mon, 21 March, 2015',
    '2018-03-12T10:12:45Z',
    '2018-06-29 17:08:00.586525+00:00',
    '2018-06-29 17:08:00.586525+05:00',
    'Tuesday , 6th September, 2017 at 4:30pm'
]

for d in date_array:
    print(f'{type(d)}: Parsing: {d}')
    dt = parse(d)
    print(type(dt))
    print(dt.date())
    print(dt.time())
    print('----')

<class 'str'>: Parsing: 2021-06-29 08:15:27.243860
<class 'datetime.datetime'>
2021-06-29
08:15:27.243860
----
<class 'str'>: Parsing: Jun 28 2018 7:40AM
<class 'datetime.datetime'>
2018-06-28
07:40:00
----
<class 'str'>: Parsing: Jun 28 2018 at 7:40AM
<class 'datetime.datetime'>
2018-06-28
07:40:00
----
<class 'str'>: Parsing: September 18, 2017, 22:19:55
<class 'datetime.datetime'>
2017-09-18
22:19:55
----
<class 'str'>: Parsing: Sun, 05/12/1999, 12:30PM
<class 'datetime.datetime'>
1999-05-12
12:30:00
----
<class 'str'>: Parsing: Mon, 21 March, 2015
<class 'datetime.datetime'>
2015-03-21
00:00:00
----
<class 'str'>: Parsing: 2018-03-12T10:12:45Z
<class 'datetime.datetime'>
2018-03-12
10:12:45
----
<class 'str'>: Parsing: 2018-06-29 17:08:00.586525+00:00
<class 'datetime.datetime'>
2018-06-29
17:08:00.586525
----
<class 'str'>: Parsing: 2018-06-29 17:08:00.586525+05:00
<class 'datetime.datetime'>
2018-06-29
17:08:00.586525
----
<class 'str'>: Parsing: Tuesday , 6th September, 2017 at 